# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686593


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<00:59,  2.12s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:00,  2.23s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:41,  1.59s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:28,  1.15s/it]

...

Rendering models:  83%|████████▎ | 25/30 [00:09<00:00,  5.56it/s]

Rendering models:  90%|█████████ | 27/30 [00:09<00:00,  6.50it/s]

Rendering models:  93%|█████████▎| 28/30 [00:10<00:00,  6.40it/s]

Rendering models:  97%|█████████▋| 29/30 [00:10<00:00,  7.04it/s]

not-logged-in-20a275b53abec8db9f41    0.007717
not-logged-in-bc5fdb9ea8d611e60b8b    0.001037
cschwefl                              0.000996
Andy_Carleen                          0.000702
djswanso                              0.001092
kjkavene                              0.000549
shocko61                              0.000761
justsomenoodles                       0.003742
jmfranci                              0.003011
not-logged-in-2908c5b97527b9f7279b    0.000746
theetick                              0.000671
sn346808                              0.021589
sn345012                              0.043442
Finn.Ralston                          0.004219
not-logged-in-641e66759c84e5fc98cf    0.816753
MajorTom69                            0.001009
not-logged-in-14d09f70263c1fe15cc3    0.000857
not-logged-in-ffc5c906399855f5ef0f    0.000779
thisguyiknow                          0.001062
Frosst.matt                           0.002515
ranyel.a.e.pasimio                    0.108208
ElisabethB   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())